# Return the One-Ip Table and devices path result
During current use case, the final goals are present the One-Ip Table of devices in current domain and path result between two random devices in current domain.
There are totally 10 REST APIs we going to concerned to be a part of this use case, as shown in following:<br> 

> 1) login API ([Step 1](#Step-1:-Calling-Login-API))<br>
> 2) get_all_accessible_tenants API ([Step 2](#Step-2:-Calling-Get-All-Accessible-Tenants-API-with-authentication-token))<br>
> 3) get_all_accessible_domains API ([Step 3](#Step-3:-Calling-Get-All-Accessible-Domains-API-with-token-and-tenantId))<br>
> 4) specify_a_working_domain API ([Step 4](#Step-4:-Calling-Specify-A-Working-Domain-API))<br>
> 5) get_all_devices API ([Step 5](#Step-5:-Calling-Get-All-Devices-API))<br>
> 6) get_one_ip_table API ([Step 6](#Step-6:-Calling-Get-One-Ip-Table))<br>
> 7) resolve_device_gateway API ([Step 7](#Step-7:-Calling-Resolve-Device-Gateway-API))<br>
> 8) calculate_path API ([Step 8](#Step-8:-Calling-Calculate-Path-API))<br>
> 9) get_patth_result API ([Step 9](#Step-9:-Calling-Get-Path-Calulation-Result-API))<br>
> 10) logout API ([Step 10](#Step-10:-Calling-Logout-API))


The sequencial of provided APIs is also the sequence of our workflow steps.<br>

***Note:*** if users want to find the path results of devices, then the step sequence must be followed from number 7) to 9). If users call these APIs with a different sequential then there would be no results or some errors would be occured. 

## Step 0: import the corresponding modules in python and some fixed input parameters
***Note:*** If users try to use this code. please remember to change the "nb_url" to users' own working url.

In [1]:
# import python modules 
import requests
import time
import urllib3
import pprint
#urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import json

nb_url = "http://192.168.28.79"
headers = {'Content-Type': 'application/json', 'Accept': 'application/json'} 

## Step 1: Calling Login API
In step 1, we calling the login API with "username" and "password" as inputs. As response we can get the authentication token as one fixed input in following APIs calling. If users get errors when calling this API please check the API documentation on [Github_login](https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/STANDARD_formate_TEST1_LOGIN_API.ipynb) 

In [49]:
username = "gdluserTest"
password = "123456"

body = {
    "username" : username,      
    "password" : password  
}

login_URL = nb_url + "/ServicesAPI/API/V1/Session"

def login(login_URL, body, headers):
    try:
        # Do the HTTP request
        response = requests.post(login_URL, headers=headers, data = json.dumps(body), verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            js = response.json()
            return (js["token"])
        else:
            return ("Get token failed! - " + str(response.text))
    except Exception as e:
        return (str(e))
    
token = login(login_URL, body, headers)
print(token) # print out the authentication token.

cc25cbea-d244-494d-b6f9-32937bdadc69


## Step 2: Calling Get All Accessible Tenants API with authentication token
After we got the token from step 1, we need to use this token as a key to find all tenants which we have the access authentication. During this step, the most important feature is to get the tenant id of the corresponding tenant which we decide to work inside. After running this API successfully, we will get the tenantId of the willing tenant which will be set as another input for next step API calling. If users want to get more details about this API or get errors when calling this API please check the API documentation on [Github_tenant](https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/STANDARD_formate_TEST1_Get_All_Asseccible_Tenants_API_Test1%20.ipynb) 

In [50]:
Accessible_tenants_url = nb_url + "/ServicesAPI/API/V1/CMDB/Tenants"

def get_all_accessible_tenants(Accessible_tenants_url, token, headers):
    headers["Token"] = token
    try:
        # Do the HTTP request
        response = requests.get(Accessible_tenants_url, headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            result = response.json()
            tenants = result["tenants"]
            tenant =  [x for x in tenants if x["tenantName"] == "Initial Tenant"] # Name of the tenant which we are going to work inside
            #tj = tenant.json()
            tenantId = tenant[0]["tenantId"]
            #ten_j = json.dumps(tenant)
            return tenantId
        else:
            return ("Get tenants failed! - " + str(response.text))
    except Exception as e: return (e)

tenantId = get_all_accessible_tenants(Accessible_tenants_url, token, headers)
print(tenantId) # print out the specified tenant id.

fb24f3f0-81a7-1929-4b8f-99106c23fa5b


## Step 3: Calling Get All Accessible Domains API with token and tenantId
In step 3, we are going to find all accessible domains in the corressponding tenant which we have got the tenantId from previous step. Similar with step 2, during current API call, we have to decide which domain we are going to work inside and get the domainId at meanwhile to prepare for next API calling. If users get errors when calling this API please check the API documentation on [Github_domain](https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/STANDARD_formate_Get_all_accessible_domains_of_a_tenant%20_Test1%20.ipynb) 

In [51]:
Accessible_domains_url = nb_url + "/ServicesAPI/API/V1/CMDB/Domains"

def get_all_accessible_domains(tenantId, token, headers):
    headers["Token"] = token
    data = {"tenantId": tenantId}
    try:
        # Do the HTTP request
        response = requests.get(Accessible_domains_url, params=data, headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            result = response.json()
            domains = result["domains"]
            domain =  [x for x in domains if x["domainName"] == "Support and Service"]# Name of the domain which we are going to work inside
            domainId = domain[0]["domainId"]
            return domainId
        else:
            return ("Get domains failed! - " + str(response.text))

    except Exception as e: print (str(e))

domainId = get_all_accessible_domains(tenantId, token, headers)
print(domainId) # Print out the specified domain Id.

850ff5e9-c639-404d-85a3-d920dbee509c


## Step 4: Calling Specify A Working Domain API
After we running this step successfully, we finally complete the full login processes which means we totally join in Netbrain System by calling APIs. Next step, we will start to use Netbrain functions formally. If users get errors when calling this API please check the API documentation on [Github_domain](https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/STANDARD_formate_Specify_a_domain_to_work_on_API_Test1%20.ipynb) 

***Note:*** Users mush complete first four steps to totally join in Netbrain system everytime after logout. If users can remember their tenantId and domainId, users can only complete Step 1 then jump to Step 4 to input the tenantId and domainId directly.

In [52]:
Specify_a_working_domain_url = nb_url + "/ServicesAPI/API/V1/Session/CurrentDomain"

def specify_a_working_domain(tenantId, domainId, Specify_a_working_domain_url, headers, token):
    headers["Token"] = token
    body = {
        "tenantId": tenantId,
        "domainId": domainId
    }
    
    try:
        # Do the HTTP request
        response = requests.put(Specify_a_working_domain_url, data=json.dumps(body), headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            result = response.json()
            return ("Working Domain Specified Successfully, with domainId: " + domainId)
            
        elif response.status_code != 200:
            return ("Login failed! - " + str(response.text))

    except Exception as e: print (str(e))

res =  specify_a_working_domain(tenantId, domainId, Specify_a_working_domain_url, headers, token)
print (res)

Working Domain Specified Successfully, with domainId: 850ff5e9-c639-404d-85a3-d920dbee509c


## Step 5: Calling Get All Devices API
As we have mention at deginning, in this use case, we are going to get One-Ip table and path result. So the precondition is the information of devices. In current step, after we calling this Api successfully, we will get information of all devices in current domain. As following, i provide a sub-step, because after calling get all devices API we will get a json file from API response, it 's include device id, device management ip, device hostname and some other information. But we only need the managment Ip as input for next step. Thus, we provide a small funtion to filt out the "mgmIp" from step5 json file.
If users get errors when calling this API please check the API documentation on [Github_devices](https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/Device%20API%20Design/STANDARD_formate_Get_Devices_API_Test.ipynb) 

In [53]:
Get_all_devices_in_domain_url = nb_url + "/ServicesAPI/API/V1/CMDB/Devices"

def get_all_devices(Get_all_devices_in_domain_url, headers, token):
    try:
        response = requests.get(Get_all_devices_in_domain_url, headers=headers, verify=False)
        if response.status_code == 200:
            result = response.json()
            devices = result["devices"]
            return devices
        else:
            return("Get Devices failed! - " + str(response.text))
    except Exception as e:
        return (str(e)) 
    
devices = get_all_devices(Get_all_devices_in_domain_url, headers, token)

print("Total Devices Number: " + str(len(devices)))

devices

Total Devices Number: 93


[{'id': '1266a178-b829-43c8-9c24-c34154a15d30',
  'mgmtIP': '192.168.28.204',
  'hostname': 'R20',
  'deviceTypeName': 'Cisco Router',
  'firstDiscoverTime': '0001-01-01T00:00:00',
  'lastDiscoverTime': '0001-01-01T00:00:00'},
 {'id': '497b25bd-1f8c-4bfa-80be-49ab692ce4d4',
  'mgmtIP': '123.10.1.10',
  'hostname': 'R3',
  'deviceTypeName': 'Cisco Router',
  'firstDiscoverTime': '0001-01-01T00:00:00',
  'lastDiscoverTime': '0001-01-01T00:00:00'},
 {'id': 'f190b385-676f-4579-ad6d-700122a21caf',
  'mgmtIP': '123.10.1.17',
  'hostname': 'R2',
  'deviceTypeName': 'Cisco Router',
  'firstDiscoverTime': '0001-01-01T00:00:00',
  'lastDiscoverTime': '0001-01-01T00:00:00'},
 {'id': '1d48d218-06cf-4657-af2c-39796946122b',
  'mgmtIP': '123.10.1.1',
  'hostname': 'R4',
  'deviceTypeName': 'Cisco Router',
  'firstDiscoverTime': '0001-01-01T00:00:00',
  'lastDiscoverTime': '0001-01-01T00:00:00'},
 {'id': '81229708-571a-419a-a10d-9481661718a4',
  'mgmtIP': '123.10.1.2',
  'hostname': 'R1',
  'deviceTy

### A sub-step of Step 5, to filt out the "mgmIp" from json file.

In [54]:
ips = []

for i in range(len(devices)):
    ip = devices[i]["mgmtIP"]
    if ip != "":
        ips.append(ip)
  
print(str(len(ips)))
ips

88


['192.168.28.204',
 '123.10.1.10',
 '123.10.1.17',
 '123.10.1.1',
 '123.10.1.2',
 '123.10.1.6',
 '123.10.1.22',
 '123.7.7.7',
 '123.8.8.8',
 '123.10.10.10',
 '123.11.11.11',
 '123.9.9.9',
 '123.203.3.3',
 '123.204.4.4',
 '123.15.15.15',
 '123.20.1.11',
 '123.20.1.3',
 '123.20.1.10',
 '123.20.1.2',
 '10.18.19.18',
 '123.12.12.12',
 '10.1.12.2',
 '123.13.13.13',
 '10.1.13.2',
 '123.14.14.14',
 '10.1.14.2',
 '10.54.221.66',
 '172.21.31.254',
 '172.21.28.33',
 '172.31.224.21',
 '172.31.83.7',
 '172.31.224.18',
 '123.19.19.19',
 '10.108.64.254',
 '10.108.64.29',
 '10.108.80.19',
 '10.108.64.23',
 '10.108.64.41',
 '10.108.64.27',
 '10.108.64.17',
 '10.108.64.33',
 '10.108.64.48',
 '162.8.250.239',
 '10.108.88.30',
 '10.108.80.41',
 '172.31.224.70',
 '172.31.224.110',
 '96.34.192.120',
 '172.31.224.130',
 '172.31.224.40',
 '172.31.224.100',
 '172.31.224.111',
 '172.31.224.121',
 '172.31.75.8',
 '172.31.64.66',
 '172.31.224.20',
 '172.31.224.131',
 '172.31.40.72',
 '172.31.40.76',
 '172.31.40.

## Step 6: Calling Get One Ip Table
After we got all devices management IP from step 5, we can calling the Get One-Ip table API. During this step, we totally call twice of corresponding API, first calling we loop all devices in current domian with full input parameters provided, second call we only provide the "beginIndex" and "count" parameters. If users want to get more details about this API or get errors when calling this API please check the API documentation on [Github_One-Ip_Table](https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/Overall%20Plan%EF%BC%88Northbound%20API%EF%BC%89/STANDARD_formate_GEt_One_Ip_Table_API_Test1%20.ipynb) 

In [55]:
Get_One_Ip_Table_url = nb_url + "/ServicesAPI/API/V1/CMDB/Topology/OneIPTable"
results_with_Ip_Input = []
results = []

def get_one_ip_table(Get_One_Ip_Table_url, ip, beginIndex, count, headers, token):
    headers["Token"] = token
    data = {
        "ip" : ip,
        "beginIndex" : beginIndex,
        "count" : count
    }
    try:
        response = requests.get(Get_One_Ip_Table_url, params = data, headers = headers, verify = False)
        if response.status_code == 200:
            result = response.json()
            return (result)
        else:
            return ("Get One-Ip Table failed! - " + str(response.text))

    except Exception as e:
        return (str(e))  

# First Calling
for i in range(len(ips)):
    result = get_one_ip_table(Get_One_Ip_Table_url, ips[i], 0, 3, headers, token)
    #res = result["OneIPList"]
    results.append(result)

print(str(len(results)))
results


88


[{'OneIPList': [{'lanSegment': '192.168.28.0/22',
    'ip': '192.168.28.204',
    'mac': 'AABB.CC00.2210',
    'devName': 'R20',
    'interfaceName': 'Ethernet0/1',
    'switchName': '',
    'portName': '',
    'alias': '',
    'dns': 'R20.Ethernet0/1',
    'sourceDevice': 'R20',
    'serverType': 2,
    'switchType': 2001,
    'updateTime': '2019-02-19T06:04:12Z',
    'userFlag': 9,
    'source': 'Device Interface',
    'vendor': '',
    'descr': ''}],
  'statusCode': 790200,
  'statusDescription': 'Success.'},
 {'OneIPList': [{'lanSegment': '123.10.1.8/30',
    'ip': '123.10.1.10',
    'mac': 'AABB.CC00.0710',
    'devName': 'R3',
    'interfaceName': 'Ethernet0/1',
    'switchName': '',
    'portName': '',
    'alias': '',
    'dns': 'R3.Ethernet0/1',
    'sourceDevice': 'R3',
    'serverType': 2,
    'switchType': 2001,
    'updateTime': '2019-02-19T06:04:13Z',
    'userFlag': 9,
    'source': 'Device Interface',
    'vendor': '',
    'descr': ''}],
  'statusCode': 790200,
  'statu

In [58]:
# Second calling
result = get_one_ip_table(Get_One_Ip_Table_url, "", 0, 3, headers, token)
temp1 = result["OneIPList"]
print(str(len(temp1)))
temp1

3


[{'lanSegment': '192.152.38.24/29',
  'ip': '192.152.38.30',
  'mac': '78BA.F929.DDA1',
  'devName': 'DTC-COREWAN-PE2',
  'interfaceName': 'Bundle-Ether5.678',
  'switchName': '',
  'portName': '',
  'alias': '',
  'dns': 'DTC-COREWAN-PE2.Bundle-Ether5.678',
  'sourceDevice': 'DTC-COREWAN-PE2',
  'serverType': 2999,
  'switchType': 2001,
  'updateTime': '2019-02-19T06:04:38Z',
  'userFlag': 9,
  'source': 'Device Interface',
  'vendor': '',
  'descr': ''},
 {'lanSegment': '96.34.14.48/31',
  'ip': '96.34.14.48',
  'mac': '8CB6.4FE9.678A',
  'devName': 'dtr01krnyne',
  'interfaceName': 'Bundle-Ether53.10',
  'switchName': '',
  'portName': '',
  'alias': '',
  'dns': 'dtr01krnyne.Bundle-Ether53.10',
  'sourceDevice': 'dtr01krnyne',
  'serverType': 2999,
  'switchType': 2001,
  'updateTime': '2019-02-19T06:04:41Z',
  'userFlag': 9,
  'source': 'Device Interface',
  'vendor': 'Cisco Systems',
  'descr': ''},
 {'lanSegment': '96.34.14.48/31',
  'ip': '96.34.14.49',
  'mac': 'D46D.5020.6B1D

## Step 7: Calling Resolve Device Gateway API
Our second goal of this use case is to get the path result of random devices in current domain. Now we start from resolve devices gateway API. Mention again, if users want to get path result by calling APIs then users must follow the sequencial of step 7 to step 9. In step 7, we will input the Ip list we have got from step 5. After the API running successfully, we will get a gateway list with some device informations which is the required input for next step. If users want to get more details about this API or get errors when calling this API please check the API documentation on [Github_Gateway](https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/Path%20API%20Design/STANDARD_formate_Resolve_Device_Gateway_API_Test.ipynb)

In [59]:
Resolve_Device_Gateway_url = nb_url + "/ServicesAPI/API/V1/CMDB/Path/Gateways"
results = []
def resolve_device_gateway(Resolve_Device_Gateway_url, token, ipOrHost, headers):
    headers["Token"] = token
    data = {"ipOrHost":ipOrHost}
    try:
        response = requests.get(Resolve_Device_Gateway_url, params = data, headers = headers, verify = False)
        if response.status_code == 200:
            result = response.json()
            return (result)
        else:
            return ("Create module attribute failed! - " + str(response.text))

    except Exception as e:
        print (str(e)) 
        
for i in range(len(ips)):
    result = resolve_device_gateway(Resolve_Device_Gateway_url, token, ips[i], headers)
    #results.append(result["gatewayList"])
    #if result["gatewayList"] != ""
    res = result["gatewayList"]
    results.append(res)
    #print (res)
#result = resolve_device_gateway(Resolve_Device_Gateway_url, token, ips[0], headers)
#result["gatewayList"]
print(str(len(results)))
results

88


[[{'ip': '192.168.28.204', 'devName': 'R20', 'intfName': 'Ethernet0/1'}],
 [{'ip': '123.10.1.10', 'devName': 'R3', 'intfName': 'Ethernet0/1'}],
 [{'ip': '123.10.1.17', 'devName': 'R2', 'intfName': 'Ethernet0/2'}],
 [{'ip': '123.10.1.1', 'devName': 'R4', 'intfName': 'Ethernet0/1'}],
 [{'ip': '123.10.1.2', 'devName': 'R1', 'intfName': 'Ethernet0/2'}],
 [{'ip': '123.10.1.6', 'devName': 'R5', 'intfName': 'Ethernet0/1'}],
 [{'ip': '123.10.1.22', 'devName': 'R6', 'intfName': 'Ethernet0/2'}],
 [{'ip': '123.7.7.7', 'devName': 'R7', 'intfName': 'Loopback0'}],
 [{'ip': '123.8.8.8', 'devName': 'R8', 'intfName': 'Loopback0'}],
 [{'ip': '123.10.10.10', 'devName': 'R10', 'intfName': 'Loopback0'}],
 [{'ip': '123.11.11.11', 'devName': 'R11', 'intfName': 'Loopback0'}],
 [{'ip': '123.9.9.9', 'devName': 'R9', 'intfName': 'Loopback0'}],
 [{'ip': '123.203.3.3', 'devName': 'SW3', 'intfName': 'Loopback0'}],
 [{'ip': '123.204.4.4', 'devName': 'SW4', 'intfName': 'Loopback0'}],
 [{'ip': '123.15.15.15', 'devName

## Step 8: Calling Calculate Path API
During this step, we are going to calling the Calculate Path API and set the gateway information list as one input(other inputs are shown in following code cell). When calling this API, users must input the required parameters correctly and follow the format of each inputs examples([Github_calPath](https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/Path%20API%20Design/STANDARD_formate_Calculate_Path_API_Test.ipynb)). After calling this API successfully, we will get the corresponding taskId of gateway information which has been put in. And the taskId is the only one required input for next step. If users want to get more details about this API or get errors when calling this API please check the API documentation on [Github_calPath](https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/Path%20API%20Design/STANDARD_formate_Calculate_Path_API_Test.ipynb).

In [35]:
from random import randint

Calculate_Path_url = nb_url + "/ServicesAPI/API/V1/CMDB/Path/Calculation"

def calculate_path(Calculate_Path_url, body, headers, token):
    headers["Token"] = token
    
    try:
        response = requests.post(Calculate_Path_url, data = json.dumps(body), headers = headers, verify = False)
        if response.status_code == 200:
            result = response.json()
            return (result["taskID"])
        else:
            return ("Create module attribute failed! - " + str(response.text))

    except Exception as e:
        return (str(e)) 

ress = []    
for i in range(len(results)):
    gw = results[i][0]

    j = randint(0, len(results))
    gw1 = results[j][0]

    if i != j:
        sourceIP = gw["ip"]
        sourcePort = 0
        sourceGwIP = gw["ip"]
        sourceGwDev = gw["devName"]
        sourceGwIntf = gw["intfName"]
        destIP = gw1["ip"]
        destPort = 0
        pathAnalysisSet = 2
        protocol = 4
        isLive = False

        body = {
                        "sourceIP" : sourceIP,                # IP address of the source device.
                        "sourcePort" : sourcePort,
                        "sourceGwDev" : sourceGwDev,          # Hostname of the gateway device.
                        "sourceGwIP" : sourceGwIP,            # Ip address of the gateway device.
                        "sourceGwIntf" : sourceGwIntf,        # Name of the gateway interface.
                        "destIP" : destIP,                    # IP address of the destination device.
                        "destPort" : destPort,
                        "pathAnalysisSet" : pathAnalysisSet,  # 1:L3 Path; 2:L2 Path; 3:L3 Active Path
                        "protocol" : protocol,                # Specify the application protocol, check online help, such as 4 for IPv4.
                        "isLive" : isLive                     # False: Current Baseline; True: Live access
            } 

        res = calculate_path(Calculate_Path_url, body, headers, token)
        ress.append(res)

print(str(len(ress)))
ress


7
86


['e0e3cd5d-7a89-4066-a624-adc46e9a96bf',
 'caec9050-a6d2-4cba-be8c-4600ac9318d2',
 'd3b8bd4f-2cf7-4afd-a643-40409a2d140f',
 '105ecc56-0035-4a98-aaad-f27180f26c13',
 'cc7237b7-a02a-45c1-a540-29e9a7853902',
 'c90a2abf-a8db-4d6a-891d-45b56d696f75',
 '3e0d04e7-18c5-4cf6-a7e5-d35c3e4298f6',
 'b8cbb197-18a2-48f5-a30c-925b25fc1f7c',
 '86b3eff3-78d7-4990-ae02-dcf138b3d722',
 '3635d87f-53b9-403c-91de-56e44a3eb447',
 '6982b17b-e4a8-4426-8dd4-66ab39ccae86',
 '036a447c-c4f5-459e-a52f-65da5073e1a5',
 '2e877bd5-f9d1-4f22-8878-4e4be15989f6',
 '9685cdcd-748c-4dc6-a74a-09f3557df5d6',
 '72621b05-f4db-4508-9e68-7d9058902e2d',
 'b84cb313-d2b3-402f-9777-5cc437bc78a5',
 '3d6b42fb-a464-45ab-ab52-e89375a1b7c4',
 '8a034f62-5069-4de3-8dba-cc5733bd5d05',
 '1cdcb0b6-d520-43f3-b679-bf479957f104',
 '2e68f259-fb65-4ef2-ab09-c79cfc49391b',
 '5e92c88e-0925-49fc-a0d2-bff8596b266f',
 '30249cb5-258f-4712-b2a3-e038ed154cfc',
 '9691333e-1956-48c6-9b59-503afa0f54c0',
 '0796eefb-23b6-4a0a-9aad-8d50494fb179',
 '30c2d182-7db5-

In [25]:
print(str(len(ress[0])))

36


## Step 9: Calling Get Path Calulation Result API
Now we attemp to the final functional step of this use case: to get the calculation result of the task which we have got the taskId in Step 8. After we running the following sample code successfully, we will finally get the path result in a json file. If users want to get more details about this API or get errors when calling this API please check the API documentation on [Github_pathRes](https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/Path%20API%20Design/STANDARD_formate_Get_Path_Calculation_Result_API_Test.ipynb). 

In [45]:
import time
#time.sleep(30)   # Delays for 5 seconds. You can also use a float value.
def get_patth_result(Get_Path_Calulation_Result_url, headers, token):
    headers["Token"] = token
    try:
        response = requests.get(Get_Path_Calulation_Result_url, headers = headers, verify = False)
        if response.status_code == 200:
            result = response.json()
            return (result.json())
        else:
            return (response.json())

    except Exception as e:
        return (str(e)) 

Final = []
for i in range(len(ress)):
    if len(ress[i]) <= 36:
        time.sleep(3)   # Delays for 5 seconds. You can also use a float value.
        Get_Path_Calulation_Result_url = nb_url + "/ServicesAPI/API/V1/CMDB/Path/Calculation/" + str(ress[i]) + "/Result"
        final = get_patth_result(Get_Path_Calulation_Result_url, headers, token)
        d = json.dumps(final)
        j = json.loads(d)
        Final.append(final)
    
Final

[{'hopList': [{'hopId': '2a07dd34-0582-4e6a-bf4b-0cbf38c7ea9f',
    'srcDeviceName': 'R20',
    'inboundInterface': 'Ethernet3/0',
    'mediaName': '',
    'dstDeviceName': 'R3',
    'outboundInterface': 'Ethernet0/3',
    'nextHopIdList': ['ee8dc147-ecb8-4b91-aefb-2d36de3b1686']},
   {'hopId': 'ee8dc147-ecb8-4b91-aefb-2d36de3b1686',
    'srcDeviceName': 'R3',
    'inboundInterface': 'Ethernet0/0.99',
    'mediaName': '102.2.123.0/30\r\nVRF:INET',
    'dstDeviceName': 'ISP',
    'outboundInterface': 'Ethernet0/0.99-102.2.123.1(INET)',
    'nextHopIdList': []}],
  'statusCode': 794008,
  'statusDescription': "Task 'e0e3cd5d-7a89-4066-a624-adc46e9a96bf' is failed"},
 {'hopList': [{'hopId': '6dbb6106-368c-4f0a-a04e-25613ba17fc0',
    'srcDeviceName': 'R3',
    'inboundInterface': 'Ethernet0/1',
    'mediaName': '123.10.1.8/30',
    'dstDeviceName': 'R2',
    'outboundInterface': 'Ethernet0/1',
    'nextHopIdList': ['7e65023d-a59c-4d3e-a8e6-2fcad9debd2d']},
   {'hopId': '7e65023d-a59c-4d3e

From the final result, we can see all tasks are failed, that is because in Step 8 we couple the gateway addresses randomly, which means that is possible for each pair of the gateways, there is no through path between them. And as a clarification, if users get the taskId from Step 8 successfully, that is not means the path is exist during gateway pairs. 

## Step 10: Calling Logout API
After we got all informations from this case, we have to logout from the Netbrain System.
If users want to get more details about this API or get errors when calling this API please check the API documentation on [Github_logout](https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/STANDARD_formate_Logout_Test1%20.ipynb). 

In [ ]:
Logout_url = nb_url + "/ServicesAPI/API/V1/Session"

def logout(Logout_url, token, headers):
    headers["token"] = token
    
    try:
        # Do the HTTP request
        response = requests.delete(Logout_url, headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            js = response.json()
            return (js)
        else:
            return ("Session logout failed! - " + str(response.text))

    except Exception as e:
        return (str(e))

logout = logout(Logout_url, token, headers)
logout

## References:
> 1) login API: https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/STANDARD_formate_TEST1_LOGIN_API.ipynb<br> 
> 2) get_all_accessible_tenants API: https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/STANDARD_formate_TEST1_Get_All_Asseccible_Tenants_API_Test1%20.ipynb<br>
> 3) get_all_accessible_domains API: https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/STANDARD_formate_Get_all_accessible_domains_of_a_tenant%20_Test1%20.ipynb<br>
> 4) specify_a_working_domain API: https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/STANDARD_formate_Specify_a_domain_to_work_on_API_Test1%20.ipynb<br>
> 5) get_all_devices API: https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/Device%20API%20Design/STANDARD_formate_Get_Devices_API_Test.ipynb<br>
> 6) get_one_ip_table API: https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/Overall%20Plan%EF%BC%88Northbound%20API%EF%BC%89/STANDARD_formate_GEt_One_Ip_Table_API_Test1%20.ipynb<br>
> 7) resolve_device_gateway API: https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/Path%20API%20Design/STANDARD_formate_Resolve_Device_Gateway_API_Test.ipynb<br>
> 8) calculate_path API: https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/Path%20API%20Design/STANDARD_formate_Calculate_Path_API_Test.ipynb<br>
> 9) get_patth_result API: https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/Path%20API%20Design/STANDARD_formate_Get_Path_Calculation_Result_API_Test.ipynb
> 10) logout API: 
https://github.com/Gongdai/Netbrain_REST_API_First_Regularization/blob/master/Netbrain_REST_API/API_test/STANDARD_formate_Logout_Test1%20.ipynb